# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
# import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]
import ray

# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# .describe(include='all', datetime_is_numeric=True)
# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'
# pd.set_option('precision', 2)
# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

In [4]:
from data_model import arrow_dataset, fsspec_storage, db
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, stacked, daily_stats
from workflows import meta
from utilities import date_fu


config = {
    'meta': {
        'symbol': 'GORO',
        'start_date': '2016-09-10',
        'end_date': '2020-11-16',
        'config_id': 'renko_v1',
    },
    'filter': {
        'mad_value_winlen': 22,
        'mad_deviation_winlen': 1111,
        'mad_k': 17,
        'jma_winlen': 7,
        'jma_power': 2,
    },
    'sampler': {
        'renko_return': 'price_jma_return',
        'renko_size': 0.1,  # for simple runs
        'renko_reveral_multiple': 2,
        'renko_range_frac': 22,
        'renko_range_min_pct_value': 0.03,  # % of symbol value enforced as min renko size
        'max_duration_td': dt.timedelta(minutes=33),
        'min_duration_td': dt.timedelta(seconds=33),
        'min_tick_count': 33,
        'add_label': True,
        'reward_ratios': list(np.arange(2, 11, 0.5)),
    }
}

In [20]:
# find open market dates
requested_open_dates = date_fu.get_open_market_dates(config['meta']['start_date'], config['meta']['end_date'])
# avialiable tick backfill dates
backfilled_dates = db.list_symbol_dates(symbol=config['meta']['symbol'], prefix='/data/trades')
# requested+avialiable tick dates
requested_backfilled_dates = list(set(backfilled_dates).intersection(requested_open_dates))
# existing dates from results store
existing_config_dates = db.list_symbol_dates(
    symbol=config['meta']['symbol'],
    prefix=f"/samples/{config['meta']['config_id']}/bar_date/"
    )
# remaining, requested, aviable, dates
final_remaining_dates = date_fu.find_remaining_dates(
    request_dates=requested_backfilled_dates,
    existing_dates=existing_config_dates,
    )
# get daily stats for symbol (for dynamic sampling)
daily_stats_df = daily_stats.get_symbol_stats(
    symbol=config['meta']['symbol'],
    start_date=final_remaining_dates[0],
    end_date=final_remaining_dates[-1],
    source='local',
    )

In [26]:
daily_stats_df.vwap_jma_lag

0     3.957404
1     3.986806
2     4.018078
3     4.052463
4     4.097748
        ...   
467   2.921955
468   2.949007
469   2.968513
470   2.983405
471   2.995865
Name: vwap_jma_lag, Length: 472, dtype: float64

# Sample tick dates

In [ ]:
ray.init(dashboard_port=1111, ignore_reinit_error=True)

# ray.shutdown()

In [ ]:
meta.tick_sampler_workflow(config)


In [ ]:
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bds, fill_col='price_vwap')

stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked_df[['price_high','price_low','price_vwap']].plot()

In [ ]:
# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)

stats_df

stats_df.describe()